In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%run ../Functions.ipynb

In [ ]:
# Directory structure
base_dir = r"D:\Om Saran\ML\adobe\dataset\closed"
categories = ['circle', 'rectangle', 'irregular', 'star','regular', 'ellipse'] 

In [ ]:
# Data storage
data = []
labels = []

# Load data
for category in categories:
    folder_path = os.path.join(base_dir, category)
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_path.endswith('.csv'):
            # Read CSV file
            points = pd.read_csv(file_path)
            data.append(points.values)  # Store points as numpy array or DataFrame
            labels.append(category)  # Store the label

In [ ]:
unique_labels = list(set(labels))

print(unique_labels)

In [ ]:
X = []
Y = []

for curve, label in zip(data, labels):
    features = extract_features(curve)
    if features.size > 0:  # Ensure that features are not empty
        X.append(features)
        Y.append(label)
# Convert lists to numpy arrays
X = np.array(X)
Y = np.array(Y)

In [ ]:
X = X.reshape(X.shape[0], -1)

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3,random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create and train the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='rbf', C=1.0, gamma='scale')  
# RBF kernel is commonly used

# Train the model
model.fit(X_train, Y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Make predictions
Y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(Y_test, Y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Print classification report
print('Classification Report:')
print(classification_report(Y_test, Y_pred))

# Print confusion matrix
print('Confusion Matrix:')
print(confusion_matrix(Y_test, Y_pred))

In [ ]:
len(data)

In [ ]:
len(labels)

In [ ]:
import numpy as np
num_classes = 6

def normalize_curve(points, img_size):
    # Extract x and y coordinates
    x = points[:, 0]
    y = points[:, 1]

    # Normalize coordinates to fit within [0, 1]
    x_min, x_max = x.min(), x.max()
    y_min, y_max = y.min(), y.max()
    
    x_normalized = (x - x_min) / (x_max - x_min)
    y_normalized = (y - y_min) / (y_max - y_min)

    # Scale coordinates to fit within the image size
    x_scaled = (x_normalized * (img_size - 1)).astype(int)
    y_scaled = (y_normalized * (img_size - 1)).astype(int)
    
    return np.vstack((x_scaled, y_scaled)).T


In [ ]:
from tensorflow.keras.utils import to_categorical

# Create a mapping from label names to integers
label_mapping = {label: idx for idx, label in enumerate(sorted(set(labels)))}
print("Label Mapping:", label_mapping)

# Convert string labels to integers
labels = np.array([label_mapping[label] for label in labels])

labels = to_categorical(labels, num_classes)  # Convert labels to one-hot encoding

In [ ]:
len(labels)

In [ ]:
labels[0]

In [ ]:
# Example usage
print(len(data))
print(type(data))
print(type(data[0]))
print(type(data[0][0]))
print(data[0][0])

In [ ]:
data = np.array([convert_points_to_image(p, 224) for p in data])

In [ ]:
fig, ax = plt.subplots()

# Display the image
ax.imshow(data[76])
# Adjust layout to add margins
fig.tight_layout(pad=0.0)  # Increase padding as needed

# Show the plot
plt.show()

In [ ]:
data.shape

In [ ]:
data = np.stack([data] * 3, axis=-1)
data.shape

In [ ]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size = 0.1,random_state=42)

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

def build_vgg16_model(num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='sigmoid')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_vgg16_model(num_classes)


In [ ]:
model.fit(X_train, Y_train, epochs=10, batch_size=16, validation_split=0.3)

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
model.save('../Models/model_closed.h5')